In [1]:
import numpy as np
import tensorflow as tf
from pandas import read_csv
from sklearn.model_selection import train_test_split

In [2]:
full_data = read_csv("ml-1m/ratings.dat", sep="::", header=None, names=["user", "item", "label"],
                     usecols=[0, 1, 2], dtype={0: np.int32, 1: np.int32, 2: np.int32})
full_data.head()

/root/anaconda3/envs/orca-demo/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,user,item,label
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [3]:
user_set = set(full_data["user"].unique())
item_set = set(full_data["item"].unique())
min_user_id = min(user_set)
max_user_id = max(user_set)
min_item_id = min(item_set)
max_item_id = max(item_set)

min_user_id, max_user_id, min_item_id, max_item_id

(1, 6040, 1, 3952)

In [4]:
train, test = train_test_split(full_data, test_size=0.2, random_state=100)

In [5]:
num_users = max_user_id + 1
num_items = max_item_id + 1

user_input = tf.keras.layers.Input(shape=(1,), dtype="int32")
item_input = tf.keras.layers.Input(shape=(1,), dtype="int32")

# Multi-layer Perceptron
mlp_embed_user = tf.keras.layers.Embedding(input_dim=num_users,
                                           output_dim=10,
                                           input_length=1)(user_input)
mlp_embed_item = tf.keras.layers.Embedding(input_dim=num_items,
                                           output_dim=10,
                                           input_length=1)(item_input)
user_latent = tf.keras.layers.Flatten()(mlp_embed_user)
item_latent = tf.keras.layers.Flatten()(mlp_embed_item)

mlp_latent = tf.keras.layers.concatenate([user_latent, item_latent], axis=1)
mlp_latent = tf.keras.layers.Dense(20, activation="relu")(mlp_latent)
mlp_latent = tf.keras.layers.Dense(10, activation="relu")(mlp_latent)

# Matrix Factorization
mf_embed_user = tf.keras.layers.Embedding(input_dim=num_users,
                                          output_dim=20,
                                          input_length=1)(user_input)
mf_embed_item = tf.keras.layers.Embedding(input_dim=num_items,
                                          output_dim=20,
                                          input_length=1)(item_input)
mf_user_flatten = tf.keras.layers.Flatten()(mf_embed_user)
mf_item_flatten = tf.keras.layers.Flatten()(mf_embed_item)

mf_latent = tf.keras.layers.multiply([mf_user_flatten, mf_item_flatten])
concated_model = tf.keras.layers.concatenate([mlp_latent, mf_latent], axis=1)
prediction = tf.keras.layers.Dense(1, activation="relu")(concated_model)

model = tf.keras.Model([user_input, item_input], prediction)
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 10)        60410       input_1[0][0]                    
_____________________________________________________________________________________________

In [6]:
optimizer = tf.keras.optimizers.Adam(1e-2)
model.compile(optimizer=optimizer, loss="mean_squared_error")
model.fit([train.user, train.item], train.label,
          batch_size=800,
          epochs=2,
          validation_data=([test.user, test.item], test.label))

Train on 800167 samples, validate on 200042 samples
Epoch 1/2


2022-05-06 09:44:25.450611: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-06 09:44:25.450642: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-06 09:44:25.450661: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aep-001): /proc/driver/nvidia/version does not exist
2022-05-06 09:44:25.451219: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-05-06 09:44:25.460716: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2700000000 Hz
2022-05-06 09:44:25.466514: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55aaf0d0b970 initialized for platform Host (this does not guarantee 

800167/800167 [==============================] - 5s 7us/sample - loss: 1.0466 - val_loss: 0.7955
Epoch 2/2
800167/800167 [==============================] - 5s 6us/sample - loss: 0.7318 - val_loss: 0.7690


In [7]:
tf.saved_model.save(model, "./model")

2022-05-06 09:44:36.589221: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./model/assets
